<a href="https://colab.research.google.com/github/savpatpanda/protein/blob/master/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/savpatpanda/protein.git
%cd protein
!ls 
!pip3 install -r requirements.txt
# to push to GitHub, File > Save a copy in GitHub

Cloning into 'protein'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 62 (delta 28), reused 30 (delta 10), pack-reused 0
Unpacking objects: 100% (62/62), done.
/content/protein
data  notebook.ipynb  plotting.py  proteinModel.py  requirements.txt
     |████████████████████████████████| 1.8MB 5.0MB/s 
     |████████████████████████████████| 11.6MB 28.8MB/s 
     |████████████████████████████████| 14.5MB 257kB/s 
     |████████████████████████████████| 2.2MB 34.9MB/s 
     |████████████████████████████████| 296kB 37.2MB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installati

In [3]:
import numpy as np
def getFractionalToCartesianRotationMatrix(a, b, c, alpha, beta, gamma):
  alpha = alpha*np.pi/180
  beta  = beta*np.pi/180
  gamma  = gamma*np.pi/180

  n2 = (np.cos(alpha) - np.cos(gamma) * np.cos(beta)) / np.sin(gamma)
  M  = np.array([ [a,0,0],
                  [b*np.cos(gamma),b*np.sin(gamma),0], 
                  [c*np.cos(beta), c*n2, c*np.sqrt(np.sin(beta)**2-n2**2)]
                 ])
  return M


In [4]:
from gemmi import cif 
from plotting import plotSpheres
import matplotlib.pyplot as plt

parseCIFNum = lambda num: float(num[:-4] if num[-1] == ')' else num)

def parseCIFFile(cifFileName):
  doc = cif.read_file("./data/185472.cif")
  block = doc.sole_block()
  fractionalSpace = [ block.find_pair("_cell_length_a"),
                      block.find_pair("_cell_length_b"),
                      block.find_pair("_cell_length_c"),
                      block.find_pair("_cell_angle_alpha"),
                      block.find_pair("_cell_angle_beta"),
                      block.find_pair("_cell_angle_gamma")]

  fractionalSpace = map(lambda x: parseCIFNum(x[1]), fractionalSpace)
  rotationMatrix = getFractionalToCartesianRotationMatrix(*fractionalSpace)
  
  x = [parseCIFNum(x) for x in list(block.find_loop("_atom_site_fract_x"))]
  y = [parseCIFNum(x) for x in list(block.find_loop("_atom_site_fract_y"))]
  z = [parseCIFNum(x) for x in list(block.find_loop("_atom_site_fract_z"))]
  coords = np.array(list(zip(x,y,z)))

  return coords, rotationMatrix
  
try:
  coords, rotationMatrix = parseCIFFile("./data/185472.cif")
  cartesian = np.matmul(coords, rotationMatrix)
  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')
  plotSpheres(cartesian * 30, ax)
  plt.show()
  
except Exception as e:
  print(e)

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [60]:
import pandas as pd

pdb = open('data/1m46.pdb')
data = [] ##residueCode, residueNumber, x, y, z, atomType - 2,5,6,7,8, 11
positions = [2,5,6,7,8,-1]

for line in pdb:
    elements = line.split()
    if elements[0] == "ATOM" and elements[2] not in ["CA", "N", "O", "C"]:
        if len(elements) < 12:
          left = elements[2][:3]
          right = elements[2][3:]
          elements[2] = left
          elements.insert(3,right)

        item = []
        for i in positions:
          if i == 5:
            item.append(int(elements[i]))
          elif i in [6,7,8]:
            item.append(float(elements[i]))
          else:
            item.append(elements[i])
          
        data.append(item)

df = pd.DataFrame(data = data, columns = ["residueCode", "residueNumber", "x", "y", "z", "atomType"])
df = df.groupby(by = ['residueNumber'])
for key, item in df:
    print(df.get_group(key), "\n\n")


  residueCode  residueNumber       x       y      z atomType
0          CB              2  11.464  74.221 -4.060        C
1          OG              2  10.941  73.219 -4.928        O 


  residueCode  residueNumber      x       y      z atomType
2          CB              3  8.433  75.418 -7.783        C 


  residueCode  residueNumber       x       y      z atomType
3          CB              4  10.563  81.056 -7.430        C
4         OG1              4  11.855  80.851 -6.846        O
5         CG2              4  10.241  82.477 -7.002        C 


   residueCode  residueNumber      x       y       z atomType
6           CB              5  5.707  79.378  -9.636        C
7           CG              5  4.669  80.061 -10.497        C
8           CD              5  4.588  79.432 -11.891        C
9           NE              5  3.888  80.310 -12.820        N
10          CZ              5  3.687  80.032 -14.098        C
11         NH1              5  4.149  78.891 -14.599        N
12        